In [4]:
import os
import zipfile
import tempfile
import re

In [7]:
# Create a temporary directory
with tempfile.TemporaryDirectory() as temp_dir:
    # Unzip the file to the temporary directory
    with zipfile.ZipFile("../../Buildstock_TMY3_FIPS.zip", 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

    # Iterate over the files in the temporary directory
    for foldername, subfolders, filenames in os.walk(temp_dir):
        for filename in filenames:
            # If the file ends with ".epw", edit it
            if filename.endswith(".epw"):
                filepath = os.path.join(foldername, filename)
                with open(filepath, 'r+') as file:
                    content = file.read()
                    # Regex match and replace to fix heating setpoint to 5°F (-15°C)
                    content = re.sub(r"Heating,1,-?\d+(\.\d+)?,-?\d+(\.\d+)?", "Heating,1,-15,-15", content)
                    # # Write the edited content back to the file
                    file.seek(0)
                    file.write(content)
                    file.truncate()

    # Rezip the directory
    with zipfile.ZipFile("../../Buildstock_TMY3_FIPS_edited.zip", 'w') as zipf:
        for foldername, subfolders, filenames in os.walk(temp_dir):
            for filename in filenames:
                filepath = os.path.join(foldername, filename)
                zipf.write(filepath, arcname=os.path.relpath(filepath, start=temp_dir))